In [ ]:
import os

import json
#Set the file we want to use for authenticating an Azure app
#The json file stores the COMPUTER_VISION_SUBSCRIPTION_KEY and COMPUTER_VISION_ENDPOINT so we don't have it in the code.
# The json file looks like this:
#{
#"COMPUTER_VISION_SUBSCRIPTION_KEY":"___the_COMPUTER_VISION_SUBSCRIPTION_KEY___",
#"COMPUTER_VISION_ENDPOINT":"___the_COMPUTER_VISION_ENDPOINT___"
#}

cv_settings_file = 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\azure_app_cv.json'
with open(cv_settings_file, "r") as read_file:
    cv_auth_data = json.load(read_file)
#print(cv_auth_data["COMPUTER_VISION_SUBSCRIPTION_KEY"])
#print(cv_auth_data["COMPUTER_VISION_ENDPOINT"])

os.environ['COMPUTER_VISION_SUBSCRIPTION_KEY']=cv_auth_data["COMPUTER_VISION_SUBSCRIPTION_KEY"]
os.environ['COMPUTER_VISION_ENDPOINT']=cv_auth_data["COMPUTER_VISION_ENDPOINT"]

In [5]:
!pip install --upgrade azure-cognitiveservices-vision-computervision

Requirement already up-to-date: azure-cognitiveservices-vision-computervision in c:\programdata\anaconda3\lib\site-packages (0.5.0)


In [6]:
# https://docs.microsoft.com/en-us/azure/cognitive-services/computer-vision/quickstarts-sdk/python-sdk
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import TextOperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import TextRecognitionMode
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time

In [7]:
# Add your Computer Vision subscription key to your environment variables.
if 'COMPUTER_VISION_SUBSCRIPTION_KEY' in os.environ:
    subscription_key = os.environ['COMPUTER_VISION_SUBSCRIPTION_KEY']
else:
    print("\nSet the COMPUTER_VISION_SUBSCRIPTION_KEY environment variable.\n**Restart your shell or IDE for changes to take effect.**")
    sys.exit()
# Add your Computer Vision endpoint to your environment variables.
if 'COMPUTER_VISION_ENDPOINT' in os.environ:
    endpoint = os.environ['COMPUTER_VISION_ENDPOINT']
else:
    print("\nSet the COMPUTER_VISION_ENDPOINT environment variable.\n**Restart your shell or IDE for changes to take effect.**")
    sys.exit()

In [8]:
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

Stop here.  Go below.

In [6]:
remote_image_url = "https://pompeiiinpictures.com/pompeiiinpictures/R5/5%2002%2015_files/image003.jpg"

In [7]:
'''
Describe an Image - remote
This example describes the contents of an image with the confidence score.
'''
print("===== Describe an image - remote =====")
# Call API
description_results = computervision_client.describe_image(remote_image_url )

# Get the captions (descriptions) from the response, with confidence level
print("Description of remote image: ")
if (len(description_results.captions) == 0):
    print("No description detected.")
else:
    for caption in description_results.captions:
        print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))

===== Describe an image - remote =====
Description of remote image: 
'a stone building that has a bench in front of a brick wall' with confidence 62.24%


In [0]:
'''
Categorize an Image - remote
This example extracts (general) categories from a remote image with a confidence score.
'''
print("===== Categorize an image - remote =====")
# Select the visual feature(s) you want.
remote_image_features = ["categories"]
# Call API with URL and features
categorize_results_remote = computervision_client.analyze_image(remote_image_url , remote_image_features)

# Print results with confidence score
print("Categories from remote image: ")
if (len(categorize_results_remote.categories) == 0):
    print("No categories detected.")
else:
    for category in categorize_results_remote.categories:
        print("'{}' with confidence {:.2f}%".format(category.name, category.score * 100))

===== Categorize an image - remote =====
Categories from remote image: 
'outdoor_stonerock' with confidence 53.12%


In [0]:
'''
Tag an Image - remote
This example returns a tag (key word) for each thing in the image.
'''
print("===== Tag an image - remote =====")
# Call API with remote image
tags_result_remote = computervision_client.tag_image(remote_image_url )

# Print results with confidence score
print("Tags in the remote image: ")
if (len(tags_result_remote.tags) == 0):
    print("No tags detected.")
else:
    for tag in tags_result_remote.tags:
        print("'{}' with confidence {:.2f}%".format(tag.name, tag.confidence * 100))

===== Tag an image - remote =====
Tags in the remote image: 
'stone' with confidence 99.21%
'rock' with confidence 99.07%
'building' with confidence 99.00%
'outdoor' with confidence 98.60%
'brick' with confidence 92.56%
'ruin' with confidence 82.91%
'ruins' with confidence 76.29%
'old' with confidence 63.80%
'sky' with confidence 54.32%
'concrete' with confidence 45.09%
'building material' with confidence 43.49%
'cement' with confidence 19.17%


In [0]:
'''
Detect Objects - remote
This example detects different kinds of objects with bounding boxes in a remote image.
'''
print("===== Detect Objects - remote =====")
# Get URL image with different objects
remote_image_url_objects = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/objects.jpg"
# Call API with URL
detect_objects_results_remote = computervision_client.detect_objects(remote_image_url_objects)

# Print detected objects results with bounding boxes
print("Detecting objects in remote image:")
if len(detect_objects_results_remote.objects) == 0:
    print("No objects detected.")
else:
    for object in detect_objects_results_remote.objects:
        print("object at location {}, {}, {}, {}".format( \
        object.rectangle.x, object.rectangle.x + object.rectangle.w, \
        object.rectangle.y, object.rectangle.y + object.rectangle.h))

===== Detect Objects - remote =====
Detecting objects in remote image:
object at location 213, 365, 85, 208
object at location 218, 402, 179, 384
object at location 238, 417, 298, 416
object at location 116, 419, 60, 386


In [0]:
# Call API with content type (landmarks) and URL
detect_domain_results_landmarks = computervision_client.analyze_image_by_domain("landmarks", remote_image_url)
print()

print("Landmarks in the remote image:")
if len(detect_domain_results_landmarks.result["landmarks"]) == 0:
    print("No landmarks detected.")
else:
    for landmark in detect_domain_results_landmarks.result["landmarks"]:
        print(landmark["name"])


Landmarks in the remote image:
No landmarks detected.


In [0]:
'''
Detect Color - remote
This example detects the different aspects of its color scheme in a remote image.
'''
print("===== Detect Color - remote =====")
# Select the feature(s) you want
remote_image_features = ["color"]
# Call API with URL and features
detect_color_results_remote = computervision_client.analyze_image(remote_image_url, remote_image_features)

# Print results of color scheme
print("Getting color scheme of the remote image: ")
print("Is black and white: {}".format(detect_color_results_remote.color.is_bw_img))
print("Accent color: {}".format(detect_color_results_remote.color.accent_color))
print("Dominant background color: {}".format(detect_color_results_remote.color.dominant_color_background))
print("Dominant foreground color: {}".format(detect_color_results_remote.color.dominant_color_foreground))
print("Dominant colors: {}".format(detect_color_results_remote.color.dominant_colors))

===== Detect Color - remote =====
Getting color scheme of the remote image: 
Is black and white: False
Accent color: 2866A3
Dominant background color: Grey
Dominant foreground color: Grey
Dominant colors: ['Grey']


In [8]:
!pip install opencv-python
!pip install imutils

  Using cached https://files.pythonhosted.org/packages/74/41/b01f308ca4a22c8c368ed4ee80ef5318efe2f221cd0024a3a0ee9df6a94d/opencv_python-4.1.2.30-cp37-cp37m-win_amd64.whl


In [9]:
!pip install mysql.connector
!pip install requests



In [10]:
!pip install requests

Restart here

In [9]:
# import the necessary packages
from imutils import paths
import argparse
import time
import sys
import cv2
import os

# hashing with OpenCV and PythonPython
def dhash(image, hashSize=8):
    # resize the input image, adding a single column (width) so we
    # can compute the horizontal gradient
    resized = cv2.resize(image, (hashSize + 1, hashSize))

    # compute the (relative) horizontal gradient between adjacent
    # column pixels
    diff = resized[:, 1:] > resized[:, :-1]

    # convert the difference image to a hash
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

def dhash(image, hashSize=8):
    # resize the input image, adding a single column (width) so we
    # can compute the horizontal gradient
    resized = cv2.resize(image, (hashSize + 1, hashSize))
 
    # compute the (relative) horizontal gradient between adjacent
    # column pixels
    diff = resized[:, 1:] > resized[:, :-1]
 
    # convert the difference image to a hash
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

In [ ]:
import requests
import shutil
from pathlib import Path
base_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\pompeii\\pompeiiinpictures'


import mysql.connector as mariadb

mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")
cursor = mariadb_connection.cursor(buffered=True)
cursor_update = mariadb_connection.cursor(buffered=True)

number_of_models_to_run = 4992
start_id = 9
end_id = start_id+number_of_models_to_run

cursor.execute("SELECT id, id_webpage, folder, img_src,  img_alt, img_height, img_width FROM webpage_images.tbl_webpage_images WHERE id>%s AND id<%s", (start_id, end_id))

for id, id_webpage, folder, img_src,  img_alt, img_height, img_width in cursor:

    img_folder = folder
    #print(img_height)
    #print(img_width)
    if(img_folder==""):
        img_url = "https://pompeiiinpictures.com/pompeiiinpictures"+"/"+img_src  
    else:
        img_url = "https://pompeiiinpictures.com"+img_folder+"/"+img_src
    print(img_url)
    
    
    remote_image_url = img_url


    # load the image for the image has later.  Load it now because it takes a bit of time
    img_local_path = img_url.replace("https://pompeiiinpictures.com/pompeiiinpictures","")
    img_local_path = img_local_path.replace("%20"," ")
    img_local_path=base_dir+img_local_path
    #print(img_local_path)
    img_file_name = img_local_path[img_local_path.rfind('/')+1:]
    img_local_folder = img_local_path[:img_local_path.rfind('/')]

    #see if the image is there already
    if not os.path.exists(img_local_path):
        img_file_name = img_local_path[img_local_path.rfind('/')+1:]
        img_local_folder = img_local_path[:img_local_path.rfind('/')]
        
        if not os.path.exists(img_local_folder):
            #os.mkdir(img_local_folder)
            path = Path(img_local_folder)
            path.mkdir(parents=True,exist_ok=True)
        #print(img_local_folder+'/'+ img_file_name)
        
        # Open the url image, set stream to True, this will return the stream content.
        resp = requests.get(img_url, stream=True)
        # Open a local file with wb ( write binary ) permission.
        local_file = open(img_local_folder+'/'+ img_file_name, 'wb')
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        resp.raw.decode_content = True
        # Copy the response stream raw data to local image file.
        shutil.copyfileobj(resp.raw, local_file)
        # Remove the image url response object.
        del resp
        
        #with open(img_local_folder+'/'+ img_file_name, 'wb') as handle:
        #    response = requests.get(img_url, stream=True)
#
#            if not response.ok:
#                print("response not ok")
#                print(response)
#
#            for block in response.iter_content(1024):
#                if not block:
#                    break#
#
#                handle.write(block) 


    # *************************
    # Describe image
    # *************************
    azure_img_desc = ""
    azure_img_desc_confidence = 0
    if(img_height>50 and img_width>50):
        '''
        Describe an Image - remote
        This example describes the contents of an image with the confidence score.
        '''
    
        # Call API
        description_results = computervision_client.describe_image(remote_image_url )

        # Get the captions (descriptions) from the response, with confidence level

        if (len(description_results.captions) == 0):
            azure_img_desc = "No description detected."
            azure_img_desc_confidence = 0
        else:
            for caption in description_results.captions:
                azure_img_desc = caption.text
                azure_img_desc_confidence = caption.confidence * 100
    else:
        azure_img_desc = "Image too small to describe."
        azure_img_desc_confidence = 0
 

    # *************************
    # Categorize image
    # *************************
    azure_img_categories="[Not processed]"
    azure_img_categories_score=""
    # All of this is not run per saving the cost of this processing for later
    #if(img_height>50 and img_width>50):    
    #    '''
    #    Categorize an Image - remote
    #    This example extracts (general) categories from a remote image with a confidence score.
    #    '''

    #    # Select the visual feature(s) you want.
    #    remote_image_features = ["categories"]
    #    # Call API with URL and features
    #    categorize_results_remote = computervision_client.analyze_image(remote_image_url , remote_image_features)
    #    if (len(categorize_results_remote.categories) == 0):
    #        azure_img_categories="No categories detected."
    #        azure_img_categories_score=""
    #    else:
    #        for category in categorize_results_remote.categories:
    #            azure_img_categories=azure_img_categories+"["+category.name+"] "
    #            azure_img_categories_score=azure_img_categories_score+"["+str(category.score * 100)+"] "
    #else:
    #    azure_img_categories="[Image too small to categorize] "
    #    azure_img_categories_score=""

    # *************************
    # Tag image
    # *************************
    azure_img_tags=""
    azure_img_tags_confidence=""
    if(img_height>50 and img_width>50):
        '''
        Tag an Image - remote
        This example returns a tag (key word) for each thing in the image.
        '''

        # Call API with remote image
        tags_result_remote = computervision_client.tag_image(remote_image_url )

        if (len(tags_result_remote.tags) == 0):
            azure_img_tags="No tags detected."
            azure_img_tags_confidence=""
        else:
            for tag in tags_result_remote.tags:
                azure_img_tags=azure_img_tags+"["+tag.name+"] "
                azure_img_tags_confidence=azure_img_tags_confidence+"["+str(tag.confidence * 100)+"] "
    else:
        azure_img_tags="[Image too small to tag] "
        azure_img_tags_confidence=""

    # *************************

    #image hash
    imageHash = 0
    #print("image hash")
    if not os.path.exists(img_local_path):
        print("Error image not loaded for image hash   " + img_local_path)
    else:         
        # load the image from disk
        path = Path(img_local_folder)
        os.chdir(path)
        hash_image = cv2.imread(img_file_name)
 
	      # if the image is None then we could not load it from disk (so
	      # skip it)
        if not hash_image is None:
            # convert the image to grayscale and compute the hash
            hash_image = cv2.cvtColor(hash_image, cv2.COLOR_BGR2GRAY)
            imageHash = dhash(hash_image)
        else:
            imageHash = 0
            print("image hash - can't find image?")
                         
    # *************************
    #update the database
    
    #update_old_salary = (
#        "UPDATE salaries SET to_date = %s "
 # "WHERE emp_no = %s AND from_date = %s")
    update_image = ("UPDATE tbl_webpage_images SET img_url = %s, azure_img_desc = %s, azure_img_desc_confidence = %s, azure_img_categories = %s, azure_img_score = %s, azure_img_tags = %s, azure_img_tags_confidence = %s, image_hash = %s WHERE id = %s;")
    # print(update_image)
    print(id)
    cursor_update.execute(update_image, (img_url, azure_img_desc, azure_img_desc_confidence, azure_img_categories, azure_img_categories_score, azure_img_tags, azure_img_tags_confidence, imageHash, id))
    mariadb_connection.commit()
    
mariadb_connection.close()


https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2010329_files/image009.jpg
10
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2010501_files/image003.jpg
11
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2010501_files/image005.jpg
12
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2010501_files/image007.jpg
13
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2010501_files/image009.jpg
14
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2010501_files/image011.jpg
15
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2010612_files/image003.jpg
16
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2010612_files/image005.jpg
17
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2010801_files/image003.jpg
18
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2010801_files/image005.jpg
19
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2010801_files/image007.jpg
20

100
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2050619_files/image003.jpg
101
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2050619_files/image005.jpg
102
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2050619_files/image007.jpg
103
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2050619_files/image009.jpg
104
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2050619_files/image011.jpg
105
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2050619_files/image013.jpg
106
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2050619_files/image015.jpg
107
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2050619_files/image017.jpg
108
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2050619_files/image019.jpg
109
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2050619_files/image021.jpg
110
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2050619_files/i

190
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2071113_files/image021.jpg
191
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2071113_files/image023.jpg
192
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2071113_files/image025.jpg
193
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2071113_files/image027.jpg
194
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2071113_files/image029.jpg
195
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2071113_files/image031.jpg
196
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2071113_files/image033.jpg
197
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2071414_files/image003.jpg
198
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2071414_files/image005.jpg
199
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2071414_files/image007.jpg
200
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2080211_files/i

280
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2090808_files/image003.jpg
281
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2090808_files/image005.jpg
282
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2090808_files/image007.jpg
283
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2090808_files/image009.jpg
284
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2090808_files/image011.jpg
285
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2091101_files/image003.jpg
286
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2091101_files/image005.jpg
287
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2091101_files/image006.jpg
288
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2091101_files/image008.jpg
289
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2091101_files/image010.jpg
290
https://pompeiiinpictures.com/pompeiiinpictures/Altars/Altar%2091101_files/i

366
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Augustus_files/image011.jpg
367
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Augustus_files/image013.jpg
368
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Augustus_files/image015.jpg
369
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Augustus_files/image017.jpg
370
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Augustus_files/image019.jpg
371
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Augustus_files/image021.jpg
372
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Augustus_files/image023.jpg
373
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Augustus_files/image025.jpg
374
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Augustus_files/image027.jpg
375
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Augustus_files/image029.jpg
376
https://pompeiiinpictures.com/pompeiiinpictures/Arches/A

452
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Forum%20NW%20Entrance_files/image017.jpg
453
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Forum%20NW%20Entrance_files/image018.jpg
454
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Forum%20NW%20Entrance_files/image020.jpg
455
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Forum%20NW%20Entrance_files/image022.jpg
456
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Forum%20NW%20Entrance_files/image024.jpg
457
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Forum%20NW%20Entrance_files/image026.jpg
458
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Forum%20NW%20Entrance_files/image028.jpg
459
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Forum%20NW%20Entrance_files/image030.jpg
460
https://pompeiiinpictures.com/pompeiiinpictures/Arches/Arch%20Holconius%20Rufus_files/image003.jpg
461
https://pompeiiinpictures.com/pompeii

533
https://pompeiiinpictures.com/pompeiiinpictures/Fountains/Fountain%2010502_files/image003.jpg
534
https://pompeiiinpictures.com/pompeiiinpictures/Fountains/Fountain%2010502_files/image005.jpg
535
https://pompeiiinpictures.com/pompeiiinpictures/Fountains/Fountain%2010502_files/image007.jpg
536
https://pompeiiinpictures.com/pompeiiinpictures/Fountains/Fountain%2010502_files/image009.jpg
537
https://pompeiiinpictures.com/pompeiiinpictures/Fountains/Fountain%2010502_files/image011.jpg
538
https://pompeiiinpictures.com/pompeiiinpictures/Fountains/Fountain%2010502_files/image013.jpg
539
https://pompeiiinpictures.com/pompeiiinpictures/Fountains/Fountain%2010901_files/image003.jpg
540
https://pompeiiinpictures.com/pompeiiinpictures/Fountains/Fountain%2010901_files/image005.jpg
541
https://pompeiiinpictures.com/pompeiiinpictures/Fountains/Fountain%2010901_files/image007.jpg
542
https://pompeiiinpictures.com/pompeiiinpictures/Fountains/Fountain%2010901_files/image009.jpg
543
https://pompeiii